## COM6012 - 2017: Practical Quiz 2

Use logistic regression to perform classification in the [spam](./files/spambase.data) dataset. To test your algorithm split the samples in two datasets, one for training (with 70% of the samples), and one for testing (with 30% of the samples). Use a regularisation parameter of 0.01 and an elastic net parameter of 0.1.

The marks will be assigned like this:
1. Loading the data: 1 Mark
2. Performing the training stage: 2 Marks
3. Performing the testing stage: 2 Marks

Provide your solution in the Notebook.


In [21]:
val sparkVersion = "2.0.1"
val scalaVersion = scala.util.Properties.versionNumberString

sparkVersion: String = "2.0.1"
scalaVersion: String = "2.11.8"

In [22]:
classpath.add(
    "org.apache.spark" %% "spark-yarn" % sparkVersion,
    "org.apache.spark" %% "spark-mllib" % sparkVersion
)

0 new artifact(s)


In [24]:
import org.apache.spark.sql.SparkSession
import org.apache.spark.ml.{Pipeline, PipelineModel}
import org.apache.spark.ml.classification.DecisionTreeClassificationModel
import org.apache.spark.ml.classification.DecisionTreeClassifier
import org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator
import org.apache.spark.ml.feature.{IndexToString, StringIndexer, VectorIndexer}
import org.apache.spark.mllib.tree.DecisionTree
import org.apache.spark.mllib.tree.model.DecisionTreeModel
import org.apache.spark.sql.types.DoubleType
import org.apache.spark.mllib.linalg.Vectors
import org.apache.spark.sql.Row
import org.apache.spark.sql.functions._
import org.apache.spark.ml.feature.VectorAssembler
import org.apache.spark.ml.classification.LogisticRegression
import org.apache.spark.ml.linalg.Vector
import org.apache.spark.ml.tuning.{CrossValidator, ParamGridBuilder}
import org.apache.spark.mllib.regression.LabeledPoint
import org.apache.spark.ml.classification.{BinaryLogisticRegressionSummary, LogisticRegression, LogisticRegressionModel}

object LR_Class {
    
    def main(args: Array[String]) {
        
        val sparkSession = SparkSession.builder
            .master("local")
            .appName("LR_Class")
            .getOrCreate()
        
        val sc = sparkSession.sparkContext
        
        import sparkSession.implicits._
val df = sparkSession.read.format("com.databricks.spark.csv")
        .option("header", "false") //reading the headers
        .option("mode", "DROPMALFORMED")
        .load("files/spambase.data").toDF
  
        val toDouble = udf[Double, Double]( _.toDouble)
   
val dfnew = df
.withColumn("a", toDouble(df("_c0")))
.withColumn("b", toDouble(df("_c1")))
.withColumn("c", toDouble(df("_c2")))
.withColumn("d", toDouble(df("_c3")))
.withColumn("e", toDouble(df("_c4")))
.withColumn("f", toDouble(df("_c5")))
.withColumn("g", toDouble(df("_c6")))
.withColumn("h", toDouble(df("_c7")))
.withColumn("i", toDouble(df("_c8")))
.withColumn("j", toDouble(df("_c9")))
.withColumn("k", toDouble(df("_c10")))
.withColumn("l", toDouble(df("_c11")))
.withColumn("m", toDouble(df("_c12")))
.withColumn("n", toDouble(df("_c13")))
.withColumn("o", toDouble(df("_c14")))
.withColumn("p", toDouble(df("_c15")))
.withColumn("q", toDouble(df("_c16")))
.withColumn("r", toDouble(df("_c17")))
.withColumn("s", toDouble(df("_c18")))
.withColumn("t", toDouble(df("_c19")))
.withColumn("u", toDouble(df("_c20")))
.withColumn("v", toDouble(df("_c21")))
.withColumn("w", toDouble(df("_c22")))
.withColumn("x", toDouble(df("_c23")))
.withColumn("y", toDouble(df("_c24")))
.withColumn("z", toDouble(df("_c25")))
.withColumn("aa", toDouble(df("_c26")))
.withColumn("bb", toDouble(df("_c27")))
.withColumn("cc", toDouble(df("_c28")))
.withColumn("dd", toDouble(df("_c29")))
.withColumn("ee", toDouble(df("_c30")))
.withColumn("ff", toDouble(df("_c31")))
.withColumn("gg", toDouble(df("_c32")))
.withColumn("hh", toDouble(df("_c33")))
.withColumn("ii", toDouble(df("_c34")))
.withColumn("jj", toDouble(df("_c35")))
.withColumn("kk", toDouble(df("_c36")))
.withColumn("ll", toDouble(df("_c37")))
.withColumn("mm", toDouble(df("_c38")))
.withColumn("nn", toDouble(df("_c39")))
.withColumn("oo", toDouble(df("_c40")))
.withColumn("pp", toDouble(df("_c41")))
.withColumn("qq", toDouble(df("_c42")))
.withColumn("rr", toDouble(df("_c43")))
.withColumn("ss", toDouble(df("_c44")))
.withColumn("tt", toDouble(df("_c45")))
.withColumn("uu", toDouble(df("_c46")))
.withColumn("vv", toDouble(df("_c47")))
.withColumn("ww", toDouble(df("_c48")))
.withColumn("xx", toDouble(df("_c49")))
.withColumn("yy", toDouble(df("_c50")))
.withColumn("zz", toDouble(df("_c51")))
.withColumn("aaa", toDouble(df("_c52")))
.withColumn("bbb", toDouble(df("_c53")))
.withColumn("ccc", toDouble(df("_c54")))
.withColumn("ddd", toDouble(df("_c55")))
.withColumn("eee", toDouble(df("_c56")))
.withColumn("label", toDouble(df("_c57")))
.select("a","b", "c", "d", "e", "f", "g", "h", "i", "j","k","l","m","n","o","p","q","r","s","t","u","v","w","x","y","z","aa","bb", "cc", "dd",
        "ee", "ff", "gg", "hh", "ii", "jj","kk","ll","mm","nn","oo","pp","qq","rr","ss","tt","uu","vv","ww","xx","y","zz","aaa","bbb","ccc","ddd","eee","label")

        //dfnew.show()
        
val assembler = new VectorAssembler()
  .setInputCols(Array("a","b", "c", "d", "e", "f", "g", "h", "i", "j","k","l","m","n","o","p","q","r","s","t","u","v","w","x","y","z","aa","bb", "cc", "dd",
        "ee", "ff", "gg", "hh", "ii", "jj","kk","ll","mm","nn","oo","pp","qq","rr","ss","tt","uu","vv","ww","xx","y","zz","aaa","bbb","ccc","ddd","eee"))
  .setOutputCol("features")

val lr = new LogisticRegression()
  .setLabelCol("label")
  .setFeaturesCol("features")
  .setElasticNetParam(0.1)
  .setRegParam(0.01)
        
// Split the data into training and test sets (30% held out for testing).
val Array(trainingData, testData) = dfnew.randomSplit(Array(0.7, 0.3))

// Pipeline.
val pipeline = new Pipeline()
  .setStages(Array(assembler, lr))

// Train model.
val model = pipeline.fit(trainingData)
 
val predictions = model.transform(testData)

// Select (prediction, true label) and compute test error.
val evaluator = new MulticlassClassificationEvaluator()
  .setLabelCol("label")
  .setPredictionCol("prediction")
  .setMetricName("accuracy")
val accuracy = evaluator.evaluate(predictions)
println("Accuracy: "  + (accuracy))


}
}
LR_Class.main(Array())

Accuracy: 0.913235294117647


import org.apache.spark.sql.SparkSession
import org.apache.spark.ml.{Pipeline, PipelineModel}
import org.apache.spark.ml.classification.DecisionTreeClassificationModel
import org.apache.spark.ml.classification.DecisionTreeClassifier
import org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator
import org.apache.spark.ml.feature.{IndexToString, StringIndexer, VectorIndexer}
import org.apache.spark.mllib.tree.DecisionTree
import org.apache.spark.mllib.tree.model.DecisionTreeModel
import org.apache.spark.sql.types.DoubleType
import org.apache.spark.mllib.linalg.Vectors
import org.apache.spark.sql.Row
import org.apache.spark.sql.functions._
import org.apache.spark.ml.feature.VectorAssembler
import org.apache.spark.ml.classification.LogisticRegression
import org.apache.spark.ml.linalg.Vector
import org.apache.spark.ml.tuning.{CrossValidator, ParamGridBuilder}
import org.apache.spark.mllib.regression.LabeledPoint
import org.apache.spark.ml.classification.{BinaryLogisticRegressionSu